# 

In [3]:
%load_ext autoreload
%autoreload 2

In [ ]:
from hydra import initialize, compose
from omegaconf import OmegaConf
from pathlib import Path

import sys 
sys.path.insert(0, "../src")

import numpy as np
from dataloader import SingleCellAndCodexDataset 
from model import FlowMatchingModelWrapper
from torch.utils.data import random_split
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer

In [2]:
with initialize(config_path="../configs/"):
    # Step 2: Compose the configuration
    config_dict = compose(config_name="train")  # replace 'config' with your actual config name

/tmp/ipykernel_554718/281137910.py:1: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path="../configs/"):


Dataset

In [3]:
dataset = SingleCellAndCodexDataset(config_dict.datamodule.rna_adata_path, 
                                        config_dict.datamodule.codex_adata_path, 
                                        config_dict.datamodule.label_columns, 
                                        config_dict.datamodule.obsm_key_rna, 
                                        config_dict.datamodule.obsm_key_codex) 

Dataloaders

In [4]:
train_data, valid_data = random_split(dataset,
                                           lengths=[0.80, 0.20])   

train_dataloader = torch.utils.data.DataLoader(train_data,
                                                batch_size=config_dict.training_config.batch_size,
                                                shuffle=True,
                                                num_workers=4)

valid_dataloader = torch.utils.data.DataLoader(valid_data,
                                                batch_size=config_dict.training_config.batch_size,
                                                shuffle=False,
                                                num_workers=4)

Model

In [5]:
model = FlowMatchingModelWrapper(input_dim=dataset.input_dim,
                                hidden_dim=config_dict.model.hidden_dim,
                                num_hidden_layers=config_dict.model.num_hidden_layers,
                                time_embedding_dim=config_dict.model.time_embedding_dim,
                                source_condition_dim=dataset.source_dim, 
                                use_batchnorm=config_dict.model.use_batchnorm,
                                sigma=config_dict.model.sigma, 
                                flavor=config_dict.model.flavor, 
                                learning_rate=config_dict.model.learning_rate, 
                                weight_decay=config_dict.model.weight_decay, 
                                distance=config_dict.model.distance)

Checkpoints

In [6]:
# Callbacks for saving checkpoints 
checkpoint_callback = ModelCheckpoint(dirpath=Path(config_dict.training_config.training_dir) / "checkpoints", 
                                        **config_dict.checkpoints)
callbacks = [checkpoint_callback]

# Logger settings 
logger = WandbLogger(save_dir=config_dict.training_config.training_dir,
                    **config_dict.logger)

trainer_generative = Trainer(callbacks=callbacks, 
                                default_root_dir=config_dict.training_config.training_dir, 
                                logger=logger,
                                **config_dict.trainer)

/home/icb/alessandro.palma/miniconda3/envs/sc_exp_design/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/alessandro.palma/miniconda3/envs/sc_exp_de ...
You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
trainer_generative.fit(
            model,
            train_dataloaders=train_dataloader,
            val_dataloaders=valid_dataloader)

You are using a CUDA device ('NVIDIA A100-PCIE-40GB MIG 3g.20gb') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: allepalma (inverse-perturbation-models) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [MIG-c1581d36-4fc7-5d77-974f-dac8cbe41b19]

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | v_mlp     | TimeConditionedMLP | 2.4 M  | train
1 | criterion | MSELoss            | 0      | train
---------------------------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.560     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]